In [7]:
%pip install torch transformers accelerate datasets evaluate numpy pandas jupyter scikit-learn tensorflow

  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached rich-14.0.0-py3-none-any.whl.metadata (18 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 MB 18.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 23.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 25.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 22.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.4/671.4 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 21.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━

# Imports


In [8]:
import os
import numpy as np
import pandas as pd
import torch
import evaluate
from datetime import datetime
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from sklearn.metrics import (
    hamming_loss, accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, average_precision_score, jaccard_score
)
import wandb
from wandb.integration.keras import WandbCallback


# Configuration and Setup

In [9]:
labels = [
    "News", "Entertainment", "Shop", "Chat", "Education",
    "Government", "Health", "Technology", "Work", "Travel", "Uncategorized"
]
num_labels = len(labels)
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

# Initialize wandb
wandb.init(
    project="URL-TITLE-classifier",
    name=f"training-run-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
)

# Model and training parameters
MODEL_PATH = "answerdotai/ModernBERT-base"
LEARNING_RATE = 2e-5
BATCH_SIZE = 16
NUM_EPOCHS = 3
UNFREEZE_ALL = True  # Set to False to use UNFREEZE_START_LAYER instead
UNFREEZE_START_LAYER = 18 
WEIGHT_DECAY = 0.01
WARMUP_RATIO = 0.1
MAX_GRAD_NORM = 1.0
SEED = 42

# Log config to wandb
wandb.config.update({
    "model_name": MODEL_PATH,
    "learning_rate": LEARNING_RATE,
    "batch_size": BATCH_SIZE,
    "num_epochs": NUM_EPOCHS,
    "num_labels": num_labels,
    "labels": labels,
    "optimizer": "AdamW",
    "weight_decay": WEIGHT_DECAY,
    "warmup_ratio": WARMUP_RATIO,
    "max_grad_norm": MAX_GRAD_NORM,
    "seed": SEED,
    "unfreeze_all": UNFREEZE_ALL,
    "unfreeze_start_layer": UNFREEZE_START_LAYER,
})

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: taimur00syed (taimur00syed-csumb) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


# Load Data

In [5]:
df = pd.read_parquet('../data/processed/cleaned_classified_data.parquet')
df = df[[df.columns[0], df.columns[1], df.columns[9]]]
df.columns = ['url', 'title', 'category']

#Create a dataset for training and validation
dataset = Dataset.from_pandas(df)
split_dataset = dataset.train_test_split(test_size=0.1, seed=1, stratify_by_column="category")
dataset_dict = DatasetDict({
    'train': split_dataset['train'],
    'validation': split_dataset['test']
})

# Log dataset info
wandb.config.update({
    "train_size": len(dataset_dict['train']),
    "val_size": len(dataset_dict['validation'])
})

# Load Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    problem_type="multi_label_classification" #This is a hardcoded value
)

#Handle freezing/unfreezing layers
if not UNFREEZE_ALL:
    for param in model.base_model.parameters():
        param.requires_grad = False
    num_layers = 22  # ModernBERT has 22 layers this is a hardcoded value
    for i in range(num_layers - UNFREEZE_START_LAYER, num_layers):
        for param in model.base_model.layers[i].parameters():
            param.requires_grad = True
    for param in model.base_model.final_norm.parameters():
        param.requires_grad = True
    else:
        for param in model.base_model.parameters():
            param.requires_grad = True

    
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
wandb.config.update({
    "trainable_parameters": trainable_params,
    "total_parameters": total_params,
    "trainable_percentage": round(trainable_params / total_params * 100, 2)
})

# Log model architecture
wandb.watch(model, log="all", log_freq=100)

# Data Preprocessing

In [ ]:
# This needs to be  [CLS][DOMAIN]{domain}[PATH]{path}[TITLE]{title}[SEP]
def preprocess_function(examples):
    texts = [f"{url}:{title}" for url, title in zip(examples['url'], examples['title'])]
    tokenized_inputs = tokenizer(texts, truncation=True, max_length=512)
    label_vectors = []
    for category in examples['category']:
        if isinstance(category, str):
            # Remove surrounding brackets (if any) then split on comma
            category_clean = category.strip("[]")
            cats = [cat.strip() for cat in category_clean.split(',')]
        elif isinstance(category, list):
            # Already a list; just strip each element
            cats = [cat.strip() for cat in category]
        else:
            cats = []
        label_vector = [1.0 if label in cats else 0.0 for label in labels]
        label_vectors.append(label_vector)
    tokenized_inputs['labels'] = label_vectors
    return tokenized_inputs
tokenized_data = dataset_dict.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Metrics

In [ ]:
def compute_metrics(eval_pred):
    logits, true_labels = eval_pred
    probabilities = 1 / (1 + np.exp(-logits))
    predictions = (probabilities >= 0.5).astype(np.int32)
    true_labels_int = true_labels.astype(np.int32)

    # Flatten arrays for micro-averaged metrics
    predictions_flat = predictions.ravel()
    true_labels_flat = true_labels_int.ravel()
    probabilities_flat = probabilities.ravel()

    # Calculate all metrics
    hamming = hamming_loss(true_labels_int, predictions)
    exact_match = accuracy_score(true_labels_int, predictions)
    
    precision_micro = precision_score(true_labels_flat, predictions_flat, average='micro')
    recall_micro = recall_score(true_labels_flat, predictions_flat, average='micro')
    f1_micro = f1_score(true_labels_flat, predictions_flat, average='micro')
    
    precision_macro = precision_score(true_labels_int, predictions, average='macro')
    recall_macro = recall_score(true_labels_int, predictions, average='macro')
    f1_macro = f1_score(true_labels_int, predictions, average='macro')
    
    precision_weighted = precision_score(true_labels_int, predictions, average='weighted')
    recall_weighted = recall_score(true_labels_int, predictions, average='weighted')
    f1_weighted = f1_score(true_labels_int, predictions, average='weighted')
    
    roc_auc_micro = roc_auc_score(true_labels_flat, probabilities_flat, average='micro')
    roc_auc_macro = roc_auc_score(true_labels_int, probabilities, average='macro', multi_class='ovr')
    
    pr_auc_micro = average_precision_score(true_labels_flat, probabilities_flat, average='micro')
    pr_auc_macro = average_precision_score(true_labels_int, probabilities, average='macro')
    
    jaccard_micro = jaccard_score(true_labels_flat, predictions_flat, average='micro')
    jaccard_macro = jaccard_score(true_labels_int, predictions, average='macro')

    metrics = {
        "hamming_loss": round(hamming, 3),
        "exact_match": round(exact_match, 3),
        "precision_micro": round(precision_micro, 3),
        "recall_micro": round(recall_micro, 3),
        "f1_micro": round(f1_micro, 3),
        "precision_macro": round(precision_macro, 3),
        "recall_macro": round(recall_macro, 3),
        "f1_macro": round(f1_macro, 3),
        "precision_weighted": round(precision_weighted, 3),
        "recall_weighted": round(recall_weighted, 3),
        "f1_weighted": round(f1_weighted, 3),
        "roc_auc_micro": round(roc_auc_micro, 3),
        "roc_auc_macro": round(roc_auc_macro, 3),
        "pr_auc_micro": round(pr_auc_micro, 3),
        "pr_auc_macro": round(pr_auc_macro, 3),
        "jaccard_micro": round(jaccard_micro, 3),
        "jaccard_macro": round(jaccard_macro, 3),
    }
    
    # Per-label metrics
    for i, label in enumerate(labels):
        metrics[f"f1_{label}"] = round(f1_score(true_labels_int[:, i], predictions[:, i], zero_division=0), 3)
        metrics[f"precision_{label}"] = round(precision_score(true_labels_int[:, i], predictions[:, i], zero_division=0), 3)
        metrics[f"recall_{label}"] = round(recall_score(true_labels_int[:, i], predictions[:, i], zero_division=0), 3)
    
    return metrics

# Training Setup

In [ ]:
class WandBCustomCallback:
    def __init__(self):
        self.train_step = 0
        self.eval_step = 0
        
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is None:
            return
            
        if "loss" in logs:
            wandb.log({
                "train/loss": logs["loss"],
                "train/learning_rate": logs["learning_rate"],
                "train/epoch": logs["epoch"],
                "train/step": self.train_step
            })
            self.train_step += 1
            
        if "eval_loss" in logs:
            metrics_dict = {f"eval/{k}": v for k, v in logs.items() if k.startswith("eval_")}
            wandb.log(metrics_dict)
            self.eval_step += 1

training_args = TrainingArguments(
    output_dir="data/models/URL-TITLE-classifier",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    logging_strategy="steps",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    report_to="wandb",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[WandBCustomCallback()],
) 

In [ ]:
torch.set_float32_matmul_precision('high')
trainer.train()

In [ ]:
for i, label in enumerate(labels):
    true_labels = np.array([example["labels"][i] for example in tokenized_data["validation"]])
    predictions = np.array([pred[i] for pred in trainer.predict(tokenized_data["validation"]).predictions])
    predictions = (predictions >= 0.5).astype(int)
    
    cm = wandb.plot.confusion_matrix(
        y_true=true_labels,
        preds=predictions,
        class_names=["Negative", "Positive"],
        title=f"Confusion Matrix - {label}"
    )
    wandb.log({f"confusion_matrix_{label}": cm})
    
example_batch = tokenized_data["validation"][:5]
predictions = trainer.predict(example_batch).predictions
for i, example in enumerate(example_batch):
    pred_probs = 1 / (1 + np.exp(-predictions[i]))
    pred_labels = (pred_probs >= 0.5).astype(int)
    
    wandb.log({
        f"example_{i}/url": example["url"],
        f"example_{i}/title": example["title"],
        f"example_{i}/true_labels": example["labels"],
        f"example_{i}/predicted_labels": pred_labels.tolist(),
        f"example_{i}/prediction_probabilities": pred_probs.tolist()
    })

# Save the model
trainer.save_model()
wandb.save("data/models/URL-TITLE-classifier/*")

In [ ]:
wandb.finish()